In [ ]:
SIGNS = {
    'king': ('K', 'k'),
    'queen': ('Q', 'q'),
    'rook': ('R', 'r'),
    'knight': ('N', 'n'),
    'bishop': ('B', 'b'),
    'pawn': ('P', 'p'),
    'empty': ('.')
} # Это словарь символов для белых и черных фигур, которые будут отображаться на доске.


def get_new_chessman(name, color): # Функция которая генерирует новую фигуру, а именно словарь с полями color и name. Color: 0 - белый, 1 - черный, -1 - для обозначения пустого поля. Name: название фигуры.
    return {
        'color': color,
        'name': name
    }


empty = get_new_chessman('empty', -1) # Создаем новую фигуру, а именно пустое поле, цвет обозначаем -1.


def fill_board(board): # Заполняем доску фигурами по правилам начальной расстановки.
    first_row = (
        'rook', 'knight', 'bishop', 'queen', 'king', 'bishop', 'knight', 'rook'
    ) 
    for index, name in enumerate(first_row):
        board[0][index] = get_new_chessman(name, 0) # На нулевую строку ставим фигуры в порядке, указанном в first_row цвета 0 (белые).
        board[7][index] = get_new_chessman(name, 1) # На седьмую строку ставим фигуры в порядке, указанном в first_row цвета 1 (черные).
        board[1][index] = get_new_chessman('pawn', 0) # На нужные строчки ставим и пешки, белые и черные.
        board[6][index] = get_new_chessman('pawn', 1)


def print_board(board): # Функция печати доски на экран.
    result = ''
    letter_row = '   A B C D E F G H\n' # Это строчка с буквами.
    result += letter_row + '\n' # Добавим ее к результирующей строке.
    for i, row in reversed(list(enumerate(board, 1))): # Пронумеруем все строчки начиния с единицы, и развернем в обратном порядке, чтобы правильно проитреироваться и вывести на экран.
        sign_row = map(
            lambda el: SIGNS[el['name']][el['color']],
            row
        ) # Получим последовательность, состоящую из символов фигур, учитывая цвет.
        result += f"{i}  {' '.join(sign_row)}  {i}\n" # Добавим ее к результирующей строке.
    result += '\n' + letter_row # Еще раз добавим строку с буквами.
    print(result) # Выведем на экран результат.


def get_color(board, x, y): # Это функция для получния цвета фигуры по координатам.
    return board[y][x]['color']


def move_chessman(board, x_from, y_from, x_to, y_to): # Это функция для перемещения фигуры из одной позиции в другую.
    board[y_to][x_to] = board[y_from][x_from]
    board[y_from][x_from] = empty


def xy_in_range(x, y): # Это функция для проверки нахождения координат в пределах доски, будет применяться при проверки корректности ходов.
    return 0 <= x < 8 and 0 <= y < 8


def get_xy(string): # Это функция для получения координат по введеной строке.
    from re import match
    if match(r'^([a-h][1-8]+)$', string) is None:
        return -1, -1 # Если координаты заданы некорретно, вернем кортеж (-1, -1)
    else: # Иначе, по букве определим x, по цифре - y.
        x = 'abcdefgh'.find(string[0])
        y = int(string[1]) - 1
    return x, y

def clear(): # Это функция для очистки экрана.
    from os import system, name
    system('cls' if name == 'nt' else 'clear')


def get_turn_message(board, move, white_turn): # Это функция для получения сообщения о номере хода и чей ход.
    return f"Ход №{move} - {'белые' if white_turn else 'черные'}."


def get_moves_king(board, x, y): # Функция получения ходов короля.
    moves = []
    for x_offset in range(-1, 2): # Перебираем все смещения от -1 до 1.
        for y_offset in range(-1, 2):
            if x_offset == 0 and y_offset == 0: # Исключаем позицию, в которой находится король.
                continue
            new_x = x + x_offset
            new_y = y + y_offset
            if xy_in_range(new_x, new_y) and get_color(board, new_x, new_y) != board[y][x]['color']: # Если координаты корретны по цвету и входят в диапазон доски, добавляем их.
                moves.append((new_x, new_y))  
    return moves


def get_moves_rook(board, x, y): # Функция получения ходов ладьи.
    moves = []
    for x_offset, y_offset in ((0, 1), (1, 0), (0, -1), (-1, 0)): # Перебираем все смещения.
        new_x = x + x_offset
        new_y = y + y_offset
        while xy_in_range(new_x, new_y) and get_color(board, new_x, new_y) == -1: # Перебираем каждое из смещений и смотрим, сколько диагоналей мы можем еще добавить, учитывая встречу любой фигуры и границ доски.
            moves.append((new_x,  new_y))
            new_x += x_offset
            new_y += y_offset
        if xy_in_range(new_x, new_y) and get_color(board, new_x, new_y) != board[y][x]['color']:
            moves.append((new_x,  new_y))
    return moves


def get_moves_knight(board, x, y): # Функция получения ходов коня.
    moves = []
    for offset1 in (-1, 1):
        for offset2 in (-2, 2): # Перебираем все смещения, смотрим на корретность цвета и границ доски.
            new_x = x + offset1
            new_y = y + offset2
            if xy_in_range(new_x, new_y) and get_color(board, new_x, new_y) != board[y][x]['color']:
                moves.append((new_x, new_y))
            new_x = x + offset2
            new_y = y + offset1
            if xy_in_range(new_x, new_y) and get_color(board, new_x, new_y) != board[y][x]['color']:
                moves.append((new_x, new_y))
    return moves


def get_moves_bishop(board, x, y): # Функция получения ходов слона.
        moves = []
        for x_offset, y_offset in ((1, 1), (1, -1), (-1, 1), (-1, -1)): # Перебираем все смещения.
            new_x = x + x_offset
            new_y = y + y_offset
            while xy_in_range(new_x, new_y) and get_color(board, new_x, new_y) == -1: # Перебираем каждое из смещений и смотрим, сколько диагоналей мы можем еще добавить, учитывая встречу любой фигуры и границ доски.
                moves.append((new_x,  new_y))
                new_x += x_offset
                new_y += y_offset
            if xy_in_range(new_x, new_y) and get_color(board, new_x, new_y) != board[y][x]['color']:
                moves.append((new_x,  new_y))
        return moves


def get_moves_pawn(board, x, y): # Функция получения ходов пешки.
    moves = []
    if board[y][x]['color'] == 0: # В зависимости от цвета, определим смещения и номер строки, с которой можно ходить на два вперед.
        y_offset1 = 1
        y_offset2 = 2
        y_first = 1
        opposite_color = 1
    else:
        y_offset1 = -1
        y_offset2 = -2
        y_first = 6
        opposite_color = 0
    new_x = x
    new_y = y + y_offset1
    if xy_in_range(new_x, new_y) and get_color(board, new_x, new_y) == -1: # Добивим координаты, если перед пешкой ничего нет
        moves.append((new_x, new_y))
        new_y = y + y_offset2
        if y == y_first and get_color(board, new_x, new_y) == -1: # Добивим координаты, если перед пешкой впереди на две клетки ничего нет, если пешка ходит первый раз
            moves.append((new_x, new_y))
    for x_offset in (-1, 1):
        new_x = x + x_offset
        new_y = y + y_offset1
        if xy_in_range(new_x, new_y) and get_color(board, new_x, new_y) == opposite_color: # Добивим координаты, если по диагонали фигура противника.
            moves.append((new_x, new_y))
    return moves


def get_moves_queen(board, x, y): # Функция получения ходов ладьи.
    moves = []
    color = board[y][x]['color']
    board[y][x] = get_new_chessman('rook', color) # Временно разместим на эту позицию ладью того же цвета, и запомним его ходы.
    moves += get_moves_rook(board, x, y)
    board[y][x] = get_new_chessman('bishop', color) # Временно разместим на эту позицию слона того же цвета, и запомним его ходы.
    moves += get_moves_bishop(board, x, y)
    board[y][x] = get_new_chessman('queen', color) # Вернем ферзь обратно.
    return moves # В результате, сложив ходы ладьи и слона, получим ходы ферзя.


def get_moves(board, x, y): # Общая функция для получения всевозможных ходов для фигуры по координатам x и y.
    d = {
        'king': get_moves_king,
        'queen': get_moves_queen,
        'rook': get_moves_rook,
        'knight': get_moves_knight,
        'bishop': get_moves_bishop,
        'pawn': get_moves_pawn,
    }
    return d[board[y][x]['name']](board, x, y) # По названию фигуры в позиции (x, y) вызывается соответвующие функции, о которых я расказывала выше.


board = [[empty] * 8 for i in range(8)] # Создаем доску, все элементы в ней - это специальные фигуры empty с цветом -1.
move = 1 # Создаем счетчик ходов.
fill_board(board) # Заполним доску фигурами.
white_turn = True # Создадим переменную для определения очереди хода.
clear() # Очистим экран.
while True:
    print_board(board) # Распечаем доску.
    print(get_turn_message(board, move, white_turn)) # Распечатаем сообщение о номере и очереди хода.
    while True: # Запросим от пользователя исходную позицию.
        x_from, y_from = get_xy(input('Введите исходную позицию вашей фигуры: '))
        if x_from == -1 or board[y_from][x_from]['color'] == -1 or not get_moves(board, x_from, y_from) or (board[y_from][x_from]['color'] == 1 and white_turn) or (board[y_from][x_from]['color'] == 0 and not white_turn): # Если координаты нельзя перевести функцией get_xy или если цвет взятой фигуры не соответвует очереди хода или если у фигуры нет возможных ходов, то сообщим о некорретности координат. 
            print('Указаны некорректные координаты')
        else:
            break
    while True: # Запросим от пользователя целевую позицию.
        x_to, y_to = get_xy(input('Введите целевую позицию вашей фигуры: '))
        if x_to == -1 or (x_to, y_to) not in get_moves(board, x_from, y_from): # Если координаты нельзя перевести функцией get_xy или если введеных координат нет в списке возможных ходов у данной фигуры, то сообщим о некорретности координат. 
            print('Указаны некорректные координаты')
        else:
            break
    move_chessman(board, x_from, y_from, x_to, y_to) # Передвинем фигуру.  
    if not white_turn: # Обновим номер ход
        move += 1
    white_turn = not white_turn # Сменим очередь ходящего.
    clear()  # Почистим экран.

   A B C D E F G H

8  r n b q k b n r  8
7  p p p p p p p p  7
6  . . . . . . . .  6
5  . . . . . . . .  5
4  . . . . . . . .  4
3  . . . . . . . .  3
2  P P P P P P P P  2
1  R N B Q K B N R  1

   A B C D E F G H

Ход №1 - белые.
Введите исходную позицию вашей фигуры: a1
Указаны некорректные координаты
Введите исходную позицию вашей фигуры: a7
Указаны некорректные координаты
Введите исходную позицию вашей фигуры: a5
Указаны некорректные координаты
Введите исходную позицию вашей фигуры: a2
Введите целевую позицию вашей фигуры: a5
Указаны некорректные координаты
Введите целевую позицию вашей фигуры: a4
   A B C D E F G H

8  r n b q k b n r  8
7  p p p p p p p p  7
6  . . . . . . . .  6
5  . . . . . . . .  5
4  P . . . . . . .  4
3  . . . . . . . .  3
2  . P P P P P P P  2
1  R N B Q K B N R  1

   A B C D E F G H

Ход №1 - черные.
Введите исходную позицию вашей фигуры: z7
Указаны некорректные координаты
Введите исходную позицию вашей фигуры: e7
Введите целевую позицию вашей фигуры: e